# SQL query from table names - Continued

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [3]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [4]:
context = [ {'role':'system', 'content':"""
 CREATE SEVERAL (3+) TABLES HERE
"""} ]



In [5]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
WRITE IN YOUR CONTEXT QUERIES HERE
"""
})

In [6]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [8]:
# new
context_user = context.copy()
print(return_CCRMSQL("Which institution has the highest average salary among its employees?", context_user))


```sql
SELECT institution_name, AVG(salary) AS avg_salary
FROM employees
GROUP BY institution_name
ORDER BY avg_salary DESC
LIMIT 1;
```


In [9]:
# old
old_context_user = old_context.copy()
print(return_CCRMSQL("Which institution has the highest average salary among its employees?", old_context_user))


This is your SQL:
```sql
SELECT Institution, AVG(salary) AS avg_salary
FROM employees
JOIN salary ON employees.ID_usr = salary.ID_usr
JOIN studies ON employees.ID_usr = studies.ID_usr
GROUP BY Institution
ORDER BY avg_salary DESC
LIMIT 1;
```

This SQL query joins the three tables (employees, salary, and studies) on the employee ID. It calculates the average salary for each institution, orders the results by the average salary in descending order, and then selects the institution with the highest average salary.


In [10]:
# new
print(return_CCRMSQL("Which institution has the highest average salary among its employees?", context_user))


```sql
SELECT institution_name, AVG(salary) AS avg_salary
FROM employees
GROUP BY institution_name
ORDER BY avg_salary DESC
LIMIT 1;
```


In [11]:
# old
print(return_CCRMSQL("Which institution has the highest average salary among its employees?", old_context_user))


This is your SQL:
```sql
SELECT e.name, AVG(s.salary) AS avg_salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
GROUP BY e.name
ORDER BY avg_salary DESC
LIMIT 1;
```

This SQL query joins the "employees" and "salary" tables on the employee ID, calculates the average salary for each employee, then selects the employee with the highest average salary.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [13]:
# new
print(return_CCRMSQL("Which employees studied at Harvard?", context_user))

# old
print(return_CCRMSQL("Which employees studied at Harvard?", old_context_user))


```sql
SELECT employee_id, employee_name
FROM employees
WHERE university = 'Harvard';
```
This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN studies s ON e.ID_usr = s.ID_usr
WHERE s.Institution = 'Harvard';
```

This SQL query selects the names of employees who studied at Harvard by joining the "employees" and "studies" tables on the ID_usr column and filtering the results to only include rows where the Institution is 'Harvard'.


In [14]:
# new
print(return_CCRMSQL("List all employees with their latest salary.", context_user))

# old
print(return_CCRMSQL("List all employees with their latest salary.", old_context_user))


```sql
SELECT e.employee_id, e.employee_name, s.salary_amount
FROM employees e
JOIN salaries s ON e.employee_id = s.employee_id
WHERE s.salary_date = (SELECT MAX(salary_date) FROM salaries WHERE employee_id = e.employee_id);
```
This is your SQL:
```sql
SELECT e.name, s.salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
WHERE s.year = (SELECT MAX(year) FROM salary WHERE ID_usr = e.ID_usr);
```

This SQL query retrieves the names of all employees along with their latest salary. It achieves this by joining the "employees" table with the "salary" table on the employee ID, then filtering the results to only include the rows where the salary year matches the maximum year for that employee.


In [15]:
# new
print(return_CCRMSQL("Find the total number of employees per institution.", context_user))

# old
print(return_CCRMSQL("Find the total number of employees per institution.", old_context_user))


```sql
SELECT institution_name, COUNT(employee_id) AS total_employees
FROM employees
GROUP BY institution_name;
```
This is your SQL:
```sql
SELECT Institution, COUNT(ID_usr) AS Total_Employees
FROM studies
GROUP BY Institution;
```

This SQL query selects the institution from the "studies" table and counts the number of employees (ID_usr) for each institution. The result is grouped by institution, showing the total number of employees per institution.
